# Importeren van data

## Importeren van vereiste libraries

In [ ]:
import os
import requests
import calendar
import json
import time
from datetime import datetime, timedelta

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## WindForecast - API - import_wind(year,month)

In [ ]:
def import_wind(year,month):
    # Bepaal het aantal dagen in de maand
    _, num_days = calendar.monthrange(year, month)

    # Map voor opslag
    output_folder = r"Data\WindForecast"
    os.makedirs(output_folder, exist_ok=True)

    # Loop over elke dag van de maand
    for day in range(1, num_days + 1):
        date_str = f"{year}-{month:02d}-{day:02d}"  # Opgelet: gebruik "-" i.p.v. "/"
        output_filename = f"WindForecast_Elia_{year}{month:02d}{day:02d}.json"
        output_path = os.path.join(output_folder, output_filename)

        if os.path.exists(output_path):
            print(f"✅ Bestand bestaat al: {output_filename}")
            continue

        print(f"⬇️ Ophalen: {output_filename}")

        url = "https://opendata.elia.be/api/explore/v2.1/catalog/datasets/ods031/records"
        all_records = []
        limit = 100
        offset = 0

        while True:
            params = {
                "order_by": "datetime",
                "limit": limit,
                "offset": offset,
                "refine": [
                    f'datetime:"{date_str}"'
                ]
            }

            response = requests.get(url, params=params)

            if response.status_code != 200:
                print(f"❌ Fout bij {date_str} (offset {offset}): {response.status_code}")
                break

            data = response.json().get("results", [])
            if not data:
                break  # Geen data meer, stop loop

            all_records.extend(data)
            offset += limit

        if all_records:
            with open(output_path, 'w', encoding='utf-8') as f:
                json.dump(all_records, f, ensure_ascii=False, indent=2)
            print(f"✅ Opgeslagen ({len(all_records)} records): {output_filename}")
        else:
            print(f"❌ Geen data voor {date_str}")

## SolarForcast - API - import_solar(year,month)

In [ ]:
def import_solar(year,month):
    # Bepaal het aantal dagen in de maand
    _, num_days = calendar.monthrange(year, month)

    # Map voor opslag
    output_folder = r"Data\SolarForecast"
    os.makedirs(output_folder, exist_ok=True)

    # Loop over elke dag van de maand
    for day in range(1, num_days + 1):
        date_str = f"{year}-{month:02d}-{day:02d}"  # Gebruik "-" in plaats van "/"
        output_filename = f"SolarForecast_Elia_{year}{month:02d}{day:02d}.json"
        output_path = os.path.join(output_folder, output_filename)

        if os.path.exists(output_path):
            print(f"✅ Bestand bestaat al: {output_filename}")
            continue

        print(f"⬇️ Ophalen: {output_filename}")

        url = "https://opendata.elia.be/api/explore/v2.1/catalog/datasets/ods032/records"
        all_records = []
        limit = 100
        offset = 0

        while True:
            params = {
                "order_by": "datetime",
                "limit": limit,
                "offset": offset,
                "refine": [
                    f'datetime:"{date_str}"',
                    'region:"Belgium"'
                ]
            }

            response = requests.get(url, params=params)

            if response.status_code != 200:
                print(f"❌ Fout bij {date_str} (offset {offset}): {response.status_code}")
                break

            data = response.json().get("results", [])
            if not data:
                break  # Geen data meer, stop loop

            all_records.extend(data)
            offset += limit

        if all_records:
            with open(output_path, 'w', encoding='utf-8') as f:
                json.dump(all_records, f, ensure_ascii=False, indent=2)
            print(f"✅ Opgeslagen ({len(all_records)} records): {output_filename}")
        else:
            print(f"❌ Geen data voor {date_str}")

## Belpex - Screenscraping - import_belpex(year, month)

In [ ]:
def import_belpex(year, month):
    # Bereken de datums voor de maand
    from_date = f"01/{month:02d}/{year}"  # Eerste dag van de maand

    # Bepaal de eerste dag van de volgende maand voor de 'until_date'
    if month == 12:
        next_month = 1
        next_year = year + 1
    else:
        next_month = month + 1
        next_year = year

    # De eerste dag van de volgende maand
    next_month_first_day = datetime(next_year, next_month, 1)
    until_date = next_month_first_day
    until_date = until_date.strftime("%d/%m/%Y")

    # Downloadpad instellen
    download_dir = os.path.join(os.getcwd(), "Data\\Belpex")
    os.makedirs(download_dir, exist_ok=True)

    # Setup voor Chrome
    options = Options()
    prefs = {
        "download.default_directory": download_dir,
        "download.prompt_for_download": False,
        "directory_upgrade": True,
        "safebrowsing.enabled": True
    }
    options.add_argument("--headless")  # Chrome wordt onzichtbaar geopend
    options.add_experimental_option("prefs", prefs)
    driver = webdriver.Chrome(options=options)


    # Niet hernoemde bestanden opkuisen
    if "BelpexFilter.csv" in os.listdir(download_dir):
        os.remove(os.path.join(download_dir, "BelpexFilter.csv"))
        print("❌ Niet hernoemde bestand BelpexFilter.csv werd verwijderd.")

    new_filename = f"Belpex_{year}{month:02d}.csv"

    # Controleer of het bestand al bestaat voor en niet van de huidige en vorige maand is
    current_year = datetime.now().year
    current_month = datetime.now().month
    recent_file = (year == current_year and (month == current_month or month == current_month - 1)) or (year == current_year - 1 and month == 12 and current_month == 1)
    if new_filename in os.listdir(download_dir) and not recent_file:
        print(f"✅ Bestandsnaam '{new_filename}' bestaat al voor {year}-{month:02d}.")
    else:
        # Een bestand van een recente maand wordt verwijderd en opnieuw gedownload
        if new_filename in os.listdir(download_dir):
            os.remove(os.path.join(download_dir, new_filename))
            print(f"❌Vorige versie van {new_filename} werd verwijderd")
        # Ga naar de website
        driver.get("https://my.elexys.be/MarketInformation/SpotBelpex.aspx")

        print(f"⬇️ Starten met het opvragen Belpex-gegevens periode {month}/{year}")
        
        # Wacht tot het formulier en de knop "Show data" beschikbaar zijn
        wait = WebDriverWait(driver, 20)
        wait.until(EC.presence_of_element_located((By.ID, "contentPlaceHolder_fromASPxDateEdit_I")))

        # Vul de datums in
        from_input = driver.find_element(By.ID, "contentPlaceHolder_fromASPxDateEdit_I")
        until_input = driver.find_element(By.ID, "contentPlaceHolder_untilASPxDateEdit_I")

        print(f"📆 Vul 'From' datum in: {from_date}")
        from_input.clear()
        from_input.send_keys(from_date)

        print(f"📆 Vul 'Until' datum in: {until_date}")
        until_input.clear()
        until_input.send_keys(until_date)

        # Klik op "Show data"
        show_data_button = driver.find_element(By.ID, "contentPlaceHolder_refreshBelpexCustomButton_I")
        print("🚀 Klik op 'Show data'")
        driver.execute_script("arguments[0].click();", show_data_button)

        # Wacht op tabelresultaten
        print("⏳ Wacht op zoekresultaten...")
        wait.until(EC.presence_of_element_located((By.ID, "contentPlaceHolder_belpexFilterGrid_DXMainTable")))
        time.sleep(5)  # Extra wachttijd voor stabiliteit

        # ✅ Klik op de juiste export-div
        print("🚀 Klik op 'Exporteer naar CSV'")
        export_button_div = wait.until(EC.element_to_be_clickable((By.ID, "ctl00_contentPlaceHolder_GridViewExportUserControl1_csvExport")))
        driver.execute_script("arguments[0].click();", export_button_div)

        # Wacht op de download
        print("⏳ Wacht op download...")
        time.sleep(5)

        # Als het bestand bestaat, hernoem het bestand naar 'Belpex_JJJJMM.csv' (bijv. Belpex_202401.csv)
        if "BelpexFilter.csv" in os.listdir(download_dir):
            new_filename = f"Belpex_{year}{month:02d}.csv"
            os.rename(os.path.join(download_dir, "BelpexFilter.csv"), os.path.join(download_dir, new_filename))
            print(f"✅ CSV gedownload en hernoemd naar: {new_filename}")
        else:
            print("❌ Geen CSV-bestand gevonden.")

        # Sluit de browser
        driver.quit()

## Oproepen importfuncties

In [ ]:
year = 2023
month = 4

import_wind(year,month)
import_solar(year,month)
import_belpex(year,month)